# Create meeting minutes from an Audio file

I downloaded some Denver City Council meeting minutes and selected a portion of the meeting for us to transcribe. You can download it here:  
https://drive.google.com/file/d/1N_kpSojRR5RYzupz6nqM8hMSoEF_R7pU/view?usp=sharing

If you'd rather work with the original data, the HuggingFace dataset is [here](https://huggingface.co/datasets/huuuyeah/meetingbank) and the audio can be downloaded [here](https://huggingface.co/datasets/huuuyeah/MeetingBank_Audio/tree/main).

The goal of this product is to use the Audio to generate meeting minutes, including actions.

For this project, you can either use the Denver meeting minutes, or you can record something of your own!

## Please note:

When you run the pip installs in the first cell below, you might get this error - it can be safely ignored - it sounds quite severe, but it doesn't seem to affect anything else in this project!


> ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.



In [1]:
!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate openai httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not curre

In [2]:
# imports

import os
import requests
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from google.colab import drive
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch

In [28]:
# Constants

AUDIO_MODEL = "whisper-1"
LLAMA = "meta-llama/Llama-3.2-3B-Instruct"

In [4]:
# New capability - connect this Colab to my Google Drive
# See immediately below this for instructions to obtain denver_extract.mp3

drive.mount("/content/drive")
audio_filename = "/content/drive/MyDrive/Colab Notebooks/llms/denver_extract.mp3"

Mounted at /content/drive


# Download denver_extract.mp3

You can either use the same file as me, the extract from Denver city council minutes, or you can try your own..

If you want to use the same as me, then please download my extract here, and put this on your Google Drive:  
https://drive.google.com/file/d/1N_kpSojRR5RYzupz6nqM8hMSoEF_R7pU/view?usp=sharing


In [23]:
# Sign in to HuggingFace Hub

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [6]:
# Sign in to OpenAI using Secrets in Colab

# openai_api_key = userdata.get('OPENAI_API_KEY')
# openai = OpenAI(api_key=openai_api_key)

In [14]:
from transformers import pipeline
transcriber = pipeline(
    task="automatic-speech-recognition",
    model="openai/whisper-medium",
    return_timestamps=True,
    device=0 if torch.cuda.is_available() else -1
)

Device set to use cuda:0


In [15]:
# Use the Whisper OpenAI model to convert the Audio to Text
# If you'd prefer to use an Open Source model, class student Youssef has contributed an open source version
# which I've added to the bottom of this colab

audio_file = open(audio_filename, "rb")
# transcription = openai.audio.transcriptions.create(model=AUDIO_MODEL, file=audio_file, response_format="text")
transcription = transcriber(audio_filename)
print(transcription)

/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


{'text': " and kind of the confluence of this whole idea of the Confluence Week, the merging of two rivers. And as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now, and it's a very big issue. So that is the reason that the back of the logo is considered water. So let me see the creation of the logo here. Yeah, so that basically kind of sums up the reason behind the logo and all the meanings behind the symbolism. And you'll hear a little bit more about our Confluence Week is basically highlighting all of these indigenous events and things that are happening around Denver so that we can kind of bring more people together and kind of share this whole idea of Indigenous Peoples Day. So thank you. Thank you so much and thanks for your leadership. All right. Welcome to the Denver City Council meeting of Monday, October 9th. Please rise with the Pledge of Allegiance by Councilman Lopez. I pledge allegiance to the fla

In [16]:
system_message = "You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown."
user_prompt = f"Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.\n{transcription}"

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]


In [17]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [29]:
tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
streamer = TextStreamer(tokenizer)
model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)
outputs = model.generate(inputs, max_new_tokens=2000, streamer=streamer)

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 16 Apr 2025

You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown.<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.
{'text': " and kind of the confluence of this whole idea of the Confluence Week, the merging of two rivers. And as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now, and it's a very big issue. So that is the reason that the back of the logo is considered water. So let me see the creation of the logo here. Yeah, so that basically kind of sums up the reason behind the logo and

In [30]:
response = tokenizer.decode(outputs[0])

In [31]:
display(Markdown(response))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 16 Apr 2025

You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown.<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.
{'text': " and kind of the confluence of this whole idea of the Confluence Week, the merging of two rivers. And as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now, and it's a very big issue. So that is the reason that the back of the logo is considered water. So let me see the creation of the logo here. Yeah, so that basically kind of sums up the reason behind the logo and all the meanings behind the symbolism. And you'll hear a little bit more about our Confluence Week is basically highlighting all of these indigenous events and things that are happening around Denver so that we can kind of bring more people together and kind of share this whole idea of Indigenous Peoples Day. So thank you. Thank you so much and thanks for your leadership. All right. Welcome to the Denver City Council meeting of Monday, October 9th. Please rise with the Pledge of Allegiance by Councilman Lopez. I pledge allegiance to the flag of the United States of America, and to the republic for which it stands, one nation under God, indivisible, with liberty and justice for all. All right, thank you, Councilman Lopez, Madam Secretary Rockall. Black. Clerk. Here. Espinosa. Here. Flynn. Gilmore? Here. Here. Cashman? Here. Canich? Here. Lopez? Here. New? Here. Ortega? Here. Sussman? Here. Mr. President? Here. 11 present. 11 members present. We do have a quorum. Approval of the minutes. Are there any corrections to the minutes of October 2nd? Seeing none, minutes of October 2nd stand approved. Council announcements. Are there any announcements by members of council? Councilman Clark. Thank you, Mr. President. I just wanted to invite everyone down to the first ever Halloween parade on Broadway in Lucky District 7. It will happen on Saturday, October 21st at 6 o'clock p.m. It'll move along Broadway from third to Alameda. It's gonna be a fun, family-friendly event. Everyone's invited to come down, wear a costume. There'll be candy for the kids, and there are tiki zombies and 29 hearse's, and all kinds of fun and funky stuff on the fun and funky part of Broadway. So please join us October 21st at six o'clock for the Broadway Halloween bread. Thank you, Mr. President. All right, thank you, Councilman Clark. I will be there. All right, presentations. Madam Secretary, do we have any presentations? None, Mr. President. Communications, do we have any communications? None, Mr. President. We do have one proclamation this evening, Proclamation 1127, an observance of the annual Indigenous Peoples Day in the city and county of Denver. Councilman Lopez, will you please read it? Thank you, Mr. President, with pride. Proclamation number 17, well, let me just say this differently, Proclamation number 1127, series of 2017, an observance of the second annual Indigenous Peoples Day in the City and County of Denver. Whereas the Council of the City and County of Denver recognizes that the indigenous peoples have lived and flourished on the lands known as the Americas since time immemorial. And that Denver and the surrounding communities are built upon the ancestral homelands of numerous indigenous tribes which include the Southern Ute, the Ute Mountain, tribes of Colorado and whereas the tribal homelands and seasonal encampments of the Arapahoe and Cheyenne people along the banks of the Cherry Creek and South Platte River confluence gave bearing to the future settlements that would become the birthplace of the Mile High City. And whereas Colorado encompasses the ancestral homelands of 48 tribes and the city and county of Denver and surrounding communities are home to the descendants of approximately 100 tribal nations. And whereas on October 3rd 2016 the City and County of Denver unanimously passed Council Bill 801 series of 2016 officially designating the second Monday of October of each year as Indigenous Peoples Day in Denver Colorado. And whereas the Council of the City and County of Denver continues to recognize and value the vast contributions made to the community through Indigenous peoples knowledge, science, philosophy, arts and culture and through these contributions the City of Denver has developed and thrived. Whereas the Indigenous community especially youth have made great efforts this year to draw attention to the contributions of Indigenous people including Confluence week, drawing record attendance to a national Indigenous youth leadership conference, leading conversations on inclusion with their peers and supporting increased Indigenous youth participation in science and engineering. Now therefore, be it proclaimed by the Council of the City and County of Denver, Section 1, that the Council of the City and County of Denver celebrates and honours the cultural and foundational contributions of Indigenous people to our history, our past, our present and future, and continues to promote the education of the Denver community about these historic and contemporary contributions of Indigenous people. Section 2, that the City and County of Denver, Colorado does hereby observe October 9, 2017 as Indigenous Peoples' Day. Section 3, that The clerk of the city and county of Denver shall attest and affix the seal of the city and county of Denver to this proclamation and that a copy be transmitted to the Denver American Indian Commission, the city and county of Denver School District Number One, and the Colorado Commission on Indian Affairs. Thank you. Councilman Lopez, your motion to adopt. Mr. President, I move that proclamation number 1127, series of 2017 be adopted. All right, it has been moved and seconded. It comes with a move to council. Councilman Lopez. Thank you, Mr. President. It gives me a lot of pleasure and pride to read this proclamation officially for the third time, but as Indigenous Peoples Day in Denver officially for the second time. It's always awesome to be able to see not just this proclamation come by my desk, but to see so many different people from our community in our council chambers. It was a very beautiful piece of artwork that you presented to us earlier, and it is exactly the spirit that we drafted this proclamation, and this actual, the ordinance that created Indigenous Peoples Day, when we sat down and wrote it, and as a community, we couldn't think of anything else to begin, except for the confluence of the two rivers. And those confluence of the two rivers created such a great city. And we live in such an amazing city. And we're all proud of it. And sometimes we, and a lot of people from all over the country or all over the world are proud of it. And sometimes a little too proud of it is telling them to go back home. But I'm kidding when I say that. But the really nice thing about this is that we are celebrating Indigenous People's Day out of pride for who we are, who we are as a city and the contributions of indigenous people to the city, not out of spite, not out of a replacement of one culture over the other, or out of contempt or disrespect. I think of a quote that Cesar Chavez made very popular, and it stuck with me for a very long time. And anytime I have the opportunity, I speak in front of children, and especially children in our community that, you know, they often second-guess themselves on where they're coming from, who they are. And I always say that, you know, it's very important to be proud of who you're from. And the quote that I use from Cesar Chavez is, you know, pride in one's own culture does does not require contempt or disrespect of another. Right? And that's very important. It's very important for us to recognize that, no matter who we are, where we come from in this society, that your pride in your own culture does not require the contempt or disrespect of another. And man, what a year for that to sit on our shoulders for a while, for us to think about. Right? And so I wanted to just to thank you all, I think the commission, there's gonna be a couple individuals that are gonna come speak. Thank you for your art, your lovely artwork, for us to see what's in your heart and what now has become, probably is gonna be a very important symbol for the community. And also just for the work, the daily work every single day. We still have a lot of brothers and sisters whose ancestors once lived in these lands freely now stand on street corners, right, in poverty, without access to services, right, without access to sobriety or even housing or jobs, and what a cruel way to pay back a culture that has paved the way for the city to be built upon its shores, right? So we have a lot of work to do, and these kind of proclamations in this day It's not a day off, it's a day on in Denver, right? And addressing those critical issues. So I know that my colleagues are very supportive. I'm gonna ask you to support this proclamation as I know you always have done in the past. I'm very proud of today. Oh, and we made Time Magazine and Newsweek once again. Today, as being a leader in terms of the cities that are celebrating Indigenous Peoples Day. I wanted to make a point out of that. Thank you, Councilman Lopez, and thank you for sponsoring this. Councilman Martega. Mr. President, I wanna ask that my name be added. I don't think I could add much more to what Councilman Lopez has shared with us. I wanna thank him for bringing this forward, and really just appreciate all the contributions that our Native American community has contributed to this great city and great state. I worked in the Lieutenant Governor's Office when the Commission on Indian Affairs was created and had the benefit of being able to go down to the Four Corners for a peace treaty signing ceremony between the Utes and the Comanches that had been sort of at odds with each other for about 100 years, and just being able to participate in that powwow was pretty awesome. And for those of you who continue to participate in the annual powwow. It's such a great opportunity for everybody else to enjoy so many of the contributions of the culture. I mean, to see that the dance continues to be carried on as well as the native language from generation to generation is just so incredible because in so many cultures, people have come here and assimilated to the norms here you know, the norms here and they lose their language and lose a lot of the culture. And in the Native community, that hasn't happened. That has, that, you know, commitment to just passing that on from generation to generation is so important. And so I'm happy to be a co-sponsor of this tonight. Thank you. All right, thank you, Councilwoman Ortega. Councilwoman Kenich. Thank you very much, and I also wanna thank my colleague for bringing this forward. And I just wanted to say a word to the artist about how beautiful and moving I thought this logo was and your description of it. And I think one of the things that is clear is, you know, the words sometimes don't convey the power of imagery or music or the other pieces that make up culture, and so I think the art is so important. And when you talked about water, I was also thinking about land, and I guess I just wanted to say thank you. Many of the Native American peoples of Colorado have been at the forefront, or actually nationally, of defending some of the public lands that have been protected over the last few years that are under attack right now. And they're places that you all, the communities have fought to protect, but that everyone gets to enjoy. And so I just think that it's an example of where cultural preservation intersects with environmental protection, with recreation and all of the other ways that public lands are so important. And so I think I just wanted to say thank you for that because I think we have some very sacred places in our country that are at risk right now. And so as we celebrate, I appreciate that there's still a piece of resistance in here and I think that I just wanna mention a solidarity. And I mentioned a feeling of solidarity with that resistance. So thank you and happy Confluence Week. Thank you, Councilman Kenich. Seeing no other comments, I'll just say a couple. In a time of such divisive ugliness and just despicable behavior from our leadership, the reason I'm so supportive of Indigenous Peoples' Day is because it means inclusivity. It means respecting all, respecting those who have been silenced on purpose for a long time and whose history has not been told. And so we celebrate inclusivity in the face of such evil times, honestly.", 'chunks': [{'timestamp': (0.0, 4.7), 'text':'and kind of the confluence of this whole idea of the Confluence Week, the merging of two'}, {'timestamp': (4.7, 5.7), 'text':'rivers.'}, {'timestamp': (5.7, 13.7), 'text': " And as we've kind of seen recently in politics and in the world, there's a lot of situations"}, {'timestamp': (13.7, 16.92), 'text': " where water is very important right now, and it's a very big issue."}, {'timestamp': (16.92, 21.94), 'text':'So that is the reason that the back of the logo is considered water.'}, {'timestamp': (21.94, 0.0), 'text': ''}, {'timestamp': (4.36, 16.36), 'text':'So let me see the creation of the logo here.'}, {'timestamp': (19.88, 24.44), 'text':'Yeah, so that basically kind of sums up the reason behind the logo and all the meanings'}, {'timestamp': (24.44, 26.0), 'text':'behind the symbolism.'}, {'timestamp': (26.0, 29.6), 'text': " And you'll hear a little bit more about our Confluence Week"}, {'timestamp': (29.6, 0.0), 'text': ''}, {'timestamp': (3.84, 5.62), 'text':'is basically highlighting all of these indigenous events and things that are happening around Denver'}, {'timestamp': (5.62, 8.2), 'text':'so that we can kind of bring more people together'}, {'timestamp': (8.2, 10.64), 'text':'and kind of share this whole idea'}, {'timestamp': (10.64, 12.16), 'text':'of Indigenous Peoples Day.'}, {'timestamp': (12.16, 13.56), 'text':'So thank you.'}, {'timestamp': (13.56, 16.68), 'text':'Thank you so much and thanks for your leadership.'}, {'timestamp': (16.68, 17.52), 'text':'All right.'}, {'timestamp': (18.44, 20.28), 'text':'Welcome to the Denver City Council meeting'}, {'timestamp': (20.28, 22.08), 'text':'of Monday, October 9th.'}, {'timestamp': (22.08, 23.6), 'text':'Please rise with the Pledge of Allegiance'}, {'timestamp': (23.6, 25.56), 'text':'by Councilman Lopez.'}, {'timestamp': (28.6, 0.0), 'text': ''}, {'timestamp': (7.0, 19.84), 'text':'I pledge allegiance to the flag of the United States of America, and to the republic for which it stands, one nation under God, indivisible, with liberty and justice for all.'}, {'timestamp': (19.84, 23.64), 'text':'All right, thank you, Councilman Lopez, Madam Secretary Rockall.'}, {'timestamp': (23.64, 24.64), 'text':'Black.'}, {'timestamp': (24.64, 25.64), 'text':'Clerk.'}, {'timestamp': (25.64, 26.64), 'text':'Here.'}, {'timestamp': (26.64, 27.64), 'text':'Espinosa.'}, {'timestamp': (27.64, 28.64), 'text':'Here.'}, {'timestamp': (28.64, 29.64), 'text':'Flynn.'}, {'timestamp': (29.64, 0.0), 'text': ''}, {'timestamp': (2.4, 3.2), 'text':'Gilmore? Here.'}, {'timestamp': (3.2, 4.4), 'text':'Here.'}, {'timestamp': (4.4, 5.0), 'text':'Cashman?'}, {'timestamp': (5.0, 5.6), 'text':'Here.'}, {'timestamp': (5.6, 6.2), 'text':'Canich?'}, {'timestamp': (6.2, 7.2), 'text':'Here.'}, {'timestamp': (7.2, 8.0), 'text':'Lopez?'}, {'timestamp': (8.0, 8.6), 'text':'Here.'}, {'timestamp': (8.6, 9.2), 'text':'New?'}, {'timestamp': (9.2, 9.6), 'text':'Here.'}, {'timestamp': (9.6, 10.2), 'text':'Ortega?'}, {'timestamp': (10.2, 10.8), 'text':'Here.'}, {'timestamp': (10.8, 11.4), 'text':'Sussman?'}, {'timestamp': (11.4, 12.0), 'text':'Here.'}, {'timestamp': (12.0, 12.8), 'text':'Mr. President?'}, {'timestamp': (12.8, 14.8), 'text':'Here.'}, {'timestamp': (14.8, 15.8), 'text':'11 present.'}, {'timestamp': (15.8, 17.0), 'text':'11 members present.'}, {'timestamp': (17.0, 18.6), 'text':'We do have a quorum.'}, {'timestamp': (18.6, 19.5), 'text':'Approval of the minutes.'}, {'timestamp': (19.5, 24.0), 'text':'Are there any corrections to the minutes of October 2nd?'}, {'timestamp': (24.0, 26.8), 'text':'Seeing none, minutes of October 2nd stand approved.'}, {'timestamp': (26.8, 27.8), 'text':'Council announcements.'}, {'timestamp': (27.8, 0.0), 'text': ''}, {'timestamp': (2.6, 7.68), 'text':'Are there any announcements by members of council?'}, {'timestamp': (8.88, 9.92), 'text':'Councilman Clark. Thank you, Mr. President.'}, {'timestamp': (9.92, 11.6), 'text':'I just wanted to invite everyone down'}, {'timestamp': (11.6, 15.64), 'text':'to the first ever Halloween parade on Broadway'}, {'timestamp': (15.64, 17.0), 'text':'in Lucky District 7.'}, {'timestamp': (17.0, 20.92), 'text': " It will happen on Saturday, October 21st at 6 o'clock p.m."}, {'timestamp': (20.92, 24.52), 'text': " It'll move along Broadway from third to Alameda."}, {'timestamp': (24.52, 26.8), 'text': " It's gonna be a fun, family-friendly event."}, {'timestamp': (26.8, 0.0), 'text': ''}, {'timestamp': (3.7, 5.42), 'text': " Everyone's invited to come down, wear a costume. There'll be candy for the kids,"}, {'timestamp': (5.42, 10.42), 'text': " and there are tiki zombies and 29 hearse's,"}, {'timestamp': (11.26, 14.3), 'text':'and all kinds of fun and funky stuff'}, {'timestamp': (14.3, 16.98), 'text':'on the fun and funky part of Broadway.'}, {'timestamp': (16.98, 19.62), 'text': " So please join us October 21st at six o'clock"}, {'timestamp': (19.62, 20.94), 'text':'for the Broadway Halloween bread.'}, {'timestamp': (20.94, 22.06), 'text':'Thank you, Mr. President.'}, {'timestamp': (22.06, 24.7), 'text':'All right, thank you, Councilman Clark.'}, {'timestamp': (24.7, 26.22), 'text':'I will be there.'}, {'timestamp': (26.22, 28.98), 'text':'All right, presentations.'}, {'timestamp': (28.98, 0.0), 'text': ''}, {'timestamp': (1.68, 2.8), 'text':'Madam Secretary, do we have any presentations? None, Mr. President.'}, {'timestamp': (2.8, 4.88), 'text':'Communications, do we have any communications?'}, {'timestamp': (4.88, 5.96), 'text':'None, Mr. President.'}, {'timestamp': (5.96, 9.16), 'text':'We do have one proclamation this evening,'}, {'timestamp': (9.16, 13.88), 'text':'Proclamation 1127, an observance of the annual'}, {'timestamp': (13.88, 17.28), 'text':'Indigenous Peoples Day in the city and county of Denver.'}, {'timestamp': (17.28, 20.2), 'text':'Councilman Lopez, will you please read it?'}, {'timestamp': (20.2, 23.2), 'text':'Thank you, Mr. President, with pride.'}, {'timestamp': (23.2, 27.24), 'text':'Proclamation number 17, well, let me just say this differently,'}, {'timestamp': (27.24, 0.0), 'text': ''}, {'timestamp': (5.72, 8.84), 'text':'Proclamation number 1127, series of 2017, an observance of the second annual Indigenous Peoples Day in the City and County of Denver.'}, {'timestamp': (8.84, 13.48), 'text':'Whereas the Council of the City and County of Denver recognizes that the indigenous peoples have lived and'}, {'timestamp': (13.48, 17.88), 'text':'flourished on the lands known as the Americas since time immemorial.'}, {'timestamp': (17.88, 23.08), 'text':'And that Denver and the surrounding communities are built upon the ancestral homelands'}, {'timestamp': (23.08, 28.12), 'text':'of numerous indigenous tribes which include the Southern Ute, the Ute Mountain,'}, {'timestamp': (28.12, 0.0), 'text': ''}, {'timestamp': (4.46, 8.78), 'text':'tribes of Colorado and whereas the tribal homelands and seasonal encampments of the Arapahoe and Cheyenne people along the banks of the Cherry Creek and South'}, {'timestamp': (8.78, 12.92), 'text':'Platte River confluence gave bearing to the future settlements that would become'}, {'timestamp': (12.92, 19.26), 'text':'the birthplace of the Mile High City. And whereas Colorado encompasses the'}, {'timestamp': (19.26, 23.14), 'text':'ancestral homelands of 48 tribes and the city and county of Denver and'}, {'timestamp': (23.14, 27.5), 'text':'surrounding communities are home to the descendants of approximately 100 tribal'}, {'timestamp': (27.5, 0.0), 'text': ''}, {'timestamp': (8.0, 14.46), 'text':'nations. And whereas on October 3rd 2016 the City and County of Denver unanimously passed Council Bill 801 series of 2016 officially designating the second Monday'}, {'timestamp': (14.46, 20.2), 'text':'of October of each year as Indigenous Peoples Day in Denver Colorado. And'}, {'timestamp': (20.2, 24.66), 'text':'whereas the Council of the City and County of Denver continues to recognize'}, {'timestamp': (24.66, 0.0), 'text': ''}, {'timestamp': (6.44, 13.4), 'text':'and value the vast contributions made to the community through Indigenous peoples knowledge, science, philosophy, arts and culture and through'}, {'timestamp': (13.4, 20.26), 'text':'these contributions the City of Denver has developed and thrived. Whereas the'}, {'timestamp': (20.26, 24.58), 'text':'Indigenous community especially youth have made great efforts this year to'}, {'timestamp': (24.58, 28.44), 'text':'draw attention to the contributions of Indigenous people including Confluence'}, {'timestamp': (28.44, 0.0), 'text': ''}, {'timestamp': (5.04, 10.56), 'text':'week, drawing record attendance to a national Indigenous youth leadership conference, leading conversations on inclusion with their peers and supporting increased Indigenous youth'}, {'timestamp': (10.56, 16.84), 'text':'participation in science and engineering. Now therefore, be it proclaimed by the Council'}, {'timestamp': (16.84, 21.6), 'text':'of the City and County of Denver, Section 1, that the Council of the City and County'}, {'timestamp': (21.6, 25.64), 'text':'of Denver celebrates and honours the cultural and foundational contributions of Indigenous'}, {'timestamp': (25.64, 0.0), 'text': ''}, {'timestamp': (7.16, 12.98), 'text':'people to our history, our past, our present and future, and continues to promote the education of the Denver community about these historic and contemporary contributions of Indigenous'}, {'timestamp': (12.98, 18.96), 'text':'people. Section 2, that the City and County of Denver,'}, {'timestamp': (18.96, 27.68), 'text': " Colorado does hereby observe October 9, 2017 as Indigenous Peoples' Day. Section 3, that"}, {'timestamp': (27.68, 0.0), 'text': ''}, {'timestamp': (3.82, 11.86), 'text':'The clerk of the city and county of Denver shall attest and affix the seal of the city and county of Denver to this proclamation and that a copy be transmitted to the Denver'}, {'timestamp': (11.86, 16.46), 'text':'American Indian Commission, the city and county of Denver School District Number One, and'}, {'timestamp': (16.46, 20.42), 'text':'the Colorado Commission on Indian Affairs.'}, {'timestamp': (20.42, 21.42), 'text':'Thank you.'}, {'timestamp': (21.42, 22.42), 'text':'Councilman Lopez, your motion to adopt.'}, {'timestamp': (22.42, 29.4), 'text':'Mr. President, I move that proclamation number 1127, series of 2017 be adopted.'}, {'timestamp': (29.4, 0.0), 'text': ''}, {'timestamp': (1.68, 2.78), 'text':'All right, it has been moved and seconded. It comes with a move to council.'}, {'timestamp': (2.78, 4.06), 'text':'Councilman Lopez.'}, {'timestamp': (4.06, 5.06), 'text':'Thank you, Mr. President.'}, {'timestamp': (6.34, 8.78), 'text':'It gives me a lot of pleasure and pride'}, {'timestamp': (8.78, 12.94), 'text':'to read this proclamation officially for the third time,'}, {'timestamp': (12.94, 15.42), 'text':'but as Indigenous Peoples Day in Denver'}, {'timestamp': (15.42, 17.06), 'text':'officially for the second time.'}, {'timestamp': (20.62, 22.58), 'text': " It's always awesome to be able to see"}, {'timestamp': (23.94, 26.62), 'text':'not just this proclamation come by my desk,'}, {'timestamp': (26.62, 28.78), 'text':'but to see so many different people'}, {'timestamp': (29.66, 0.0), 'text': ''}, {'timestamp': (3.92, 5.52), 'text':'from our community in our council chambers.'}, {'timestamp': (8.04, 9.8), 'text':'It was a very beautiful piece of artwork that you presented to us earlier,'}, {'timestamp': (9.8, 11.44), 'text':'and it is exactly the spirit'}, {'timestamp': (11.44, 14.06), 'text':'that we drafted this proclamation,'}, {'timestamp': (14.06, 17.04), 'text':'and this actual, the ordinance that created'}, {'timestamp': (17.04, 19.36), 'text':'Indigenous Peoples Day, when we sat down and wrote it,'}, {'timestamp': (19.36, 23.92), 'text': " and as a community, we couldn't think of anything else"}, {'timestamp': (23.92, 27.0), 'text':'to begin, except for the confluence of the two rivers.'}, {'timestamp': (27.0, 28.58), 'text':'And those confluence of the two rivers'}, {'timestamp': (28.58, 0.0), 'text': ''}, {'timestamp': (2.5, 4.46), 'text':'created such a great city. And we live in such an amazing city.'}, {'timestamp': (4.46, 6.38), 'text': " And we're all proud of it."}, {'timestamp': (6.38, 9.44), 'text':'And sometimes we, and a lot of people'}, {'timestamp': (9.44, 11.34), 'text':'from all over the country or all over the world'}, {'timestamp': (11.34, 12.18), 'text':'are proud of it.'}, {'timestamp': (12.18, 13.62), 'text':'And sometimes a little too proud of it'}, {'timestamp': (13.62, 15.1), 'text':'is telling them to go back home.'}, {'timestamp': (15.1, 20.1), 'text': " But I'm kidding when I say that."}, {'timestamp': (21.46, 23.48), 'text':'But the really nice thing about this'}, {'timestamp': (23.48, 27.04), 'text': " is that we are celebrating Indigenous People's Day"}, {'timestamp': (27.04, 0.0), 'text': ''}, {'timestamp': (3.86, 7.02), 'text':'out of pride for who we are, who we are as a city and the contributions of indigenous people to the city,'}, {'timestamp': (7.02, 10.56), 'text':'not out of spite, not out of a replacement'}, {'timestamp': (10.56, 11.72), 'text':'of one culture over the other,'}, {'timestamp': (11.72, 16.72), 'text':'or out of contempt or disrespect.'}, {'timestamp': (18.32, 23.32), 'text':'I think of a quote that Cesar Chavez made very popular,'}, {'timestamp': (24.24, 27.24), 'text':'and it stuck with me for a very long time.'}, {'timestamp': (27.24, 0.0), 'text': ''}, {'timestamp': (6.98, 14.02), 'text': " And anytime I have the opportunity, I speak in front of children, and especially children in our community that, you know, they often second-guess themselves on where they're coming"}, {'timestamp': (14.02, 16.28), 'text':'from, who they are.'}, {'timestamp': (16.28, 21.6), 'text': " And I always say that, you know, it's very important to be proud of who you're from."}, {'timestamp': (21.6, 26.9), 'text': " And the quote that I use from Cesar Chavez is, you know, pride in one's own culture does"}, {'timestamp': (26.9, 0.0), 'text': ''}, {'timestamp': (3.8, 4.84), 'text':'does not require contempt or disrespect of another. Right?'}, {'timestamp': (4.84, 6.52), 'text': " And that's very important."}, {'timestamp': (6.52, 8.56), 'text': " It's very important for us to recognize that,"}, {'timestamp': (8.56, 11.92), 'text':'no matter who we are, where we come from in this society,'}, {'timestamp': (11.92, 13.76), 'text':'that your pride in your own culture'}, {'timestamp': (13.76, 17.96), 'text':'does not require the contempt or disrespect of another.'}, {'timestamp': (17.96, 22.4), 'text':'And man, what a year for that to sit on our shoulders'}, {'timestamp': (22.4, 24.76), 'text':'for a while, for us to think about.'}, {'timestamp': (24.76, 25.72), 'text':'Right?'}, {'timestamp': (25.72, 0.0), 'text': ''}, {'timestamp': (5.0, 6.88), 'text':'And so I wanted to just to thank you all, I think the commission,'}, {'timestamp': (6.88, 8.28), 'text': " there's gonna be a couple individuals"}, {'timestamp': (8.28, 9.28), 'text':'that are gonna come speak.'}, {'timestamp': (9.28, 11.36), 'text':'Thank you for your art, your lovely artwork,'}, {'timestamp': (11.36, 13.8), 'text': " for us to see what's in your heart"}, {'timestamp': (13.8, 15.6), 'text':'and what now has become,'}, {'timestamp': (15.6, 18.6), 'text':'probably is gonna be a very important symbol'}, {'timestamp': (18.6, 20.12), 'text':'for the community.'}, {'timestamp': (20.12, 21.76), 'text':'And also just for the work,'}, {'timestamp': (21.76, 24.04), 'text':'the daily work every single day.'}, {'timestamp': (24.04, 25.8), 'text':'We still have a lot of brothers and sisters'}, {'timestamp': (25.8, 29.22), 'text':'whose ancestors once lived in these lands freely'}, {'timestamp': (29.22, 0.0), 'text': ''}, {'timestamp': (5.0, 5.1), 'text':'now stand on street corners, right, in poverty,'}, {'timestamp': (8.24, 12.6), 'text':'without access to services, right, without access to sobriety or even housing or jobs,'}, {'timestamp': (12.6, 17.6), 'text':'and what a cruel way to pay back a culture'}, {'timestamp': (18.4, 20.38), 'text':'that has paved the way for the city'}, {'timestamp': (20.38, 22.64), 'text':'to be built upon its shores, right?'}, {'timestamp': (22.64, 26.04), 'text':'So we have a lot of work to do,'}, {'timestamp': (26.04, 28.44), 'text':'and these kind of proclamations in this day'}, {'timestamp': (28.44, 0.0), 'text': ''}, {'timestamp': (3.4, 5.94), 'text': " It's not a day off, it's a day on in Denver, right? And addressing those critical issues."}, {'timestamp': (5.94, 10.94), 'text':'So I know that my colleagues are very supportive.'}, {'timestamp': (11.16, 13.24), 'text': " I'm gonna ask you to support this proclamation"}, {'timestamp': (13.24, 15.4), 'text':'as I know you always have done in the past.'}, {'timestamp': (15.4, 17.42), 'text': " I'm very proud of today."}, {'timestamp': (17.42, 20.96), 'text':'Oh, and we made Time Magazine and Newsweek once again.'}, {'timestamp': (20.96, 23.28), 'text':'Today, as being a leader in terms of the cities'}, {'timestamp': (23.28, 26.16), 'text':'that are celebrating Indigenous Peoples Day.'}, {'timestamp': (26.16, 29.06), 'text':'I wanted to make a point out of that.'}, {'timestamp': (29.06, 0.0), 'text': ''}, {'timestamp': (1.8, 3.5), 'text':'Thank you, Councilman Lopez, and thank you for sponsoring this.'}, {'timestamp': (3.5, 4.92), 'text':'Councilman Martega.'}, {'timestamp': (4.92, 6.82), 'text':'Mr. President, I wanna ask that my name be added.'}, {'timestamp': (6.82, 9.68), 'text': " I don't think I could add much more"}, {'timestamp': (9.68, 14.16), 'text':'to what Councilman Lopez has shared with us.'}, {'timestamp': (14.16, 16.76), 'text':'I wanna thank him for bringing this forward,'}, {'timestamp': (16.76, 19.96), 'text':'and really just appreciate all the contributions'}, {'timestamp': (19.96, 24.92), 'text':'that our Native American community has contributed'}, {'timestamp': (24.92, 28.4), 'text':'to this great city and great state.'}, {'timestamp': (28.4, 0.0), 'text': ''}, {'timestamp': (2.4, 5.24), 'text': " I worked in the Lieutenant Governor's Office when the Commission on Indian Affairs was created"}, {'timestamp': (5.24, 8.28), 'text':'and had the benefit of being able to go down'}, {'timestamp': (8.28, 13.28), 'text':'to the Four Corners for a peace treaty signing ceremony'}, {'timestamp': (13.68, 15.68), 'text':'between the Utes and the Comanches'}, {'timestamp': (15.68, 18.06), 'text':'that had been sort of at odds with each other'}, {'timestamp': (18.06, 19.8), 'text':'for about 100 years,'}, {'timestamp': (19.8, 22.4), 'text':'and just being able to participate in that powwow'}, {'timestamp': (22.4, 23.44), 'text':'was pretty awesome.'}, {'timestamp': (25.06, 28.2), 'text':'And for those of you who continue to participate'}, {'timestamp': (28.2, 0.0), 'text': ''}, {'timestamp': (1.76, 6.0), 'text': " in the annual powwow. It's such a great opportunity for everybody else"}, {'timestamp': (6.0, 10.08), 'text':'to enjoy so many of the contributions of the culture.'}, {'timestamp': (10.08, 13.92), 'text':'I mean, to see that the dance continues to be carried on'}, {'timestamp': (13.92, 16.12), 'text':'as well as the native language'}, {'timestamp': (16.12, 19.68), 'text':'from generation to generation is just so incredible'}, {'timestamp': (19.68, 22.36), 'text':'because in so many cultures,'}, {'timestamp': (22.36, 27.36), 'text':'people have come here and assimilated to the norms here'}, {'timestamp': (29.78, 0.0), 'text': ''}, {'timestamp': (4.04, 6.88), 'text':'you know, the norms here and they lose their language and lose a lot of the culture.'}, {'timestamp': (6.88, 10.48), 'text': " And in the Native community, that hasn't happened."}, {'timestamp': (10.48, 13.08), 'text':'That has, that, you know,'}, {'timestamp': (15.4, 17.86), 'text':'commitment to just passing that on'}, {'timestamp': (17.86, 21.78), 'text':'from generation to generation is so important.'}, {'timestamp': (21.78, 25.08), 'text': " And so I'm happy to be a co-sponsor of this tonight."}, {'timestamp': (25.08, 25.92), 'text':'Thank you.'}, {'timestamp': (27.56, 29.2), 'text':'All right, thank you, Councilwoman Ortega.'}, {'timestamp': (29.2, 0.0), 'text': ''}, {'timestamp': (1.72, 4.24), 'text':'Councilwoman Kenich. Thank you very much, and I also wanna thank my colleague'}, {'timestamp': (4.24, 5.68), 'text':'for bringing this forward.'}, {'timestamp': (5.68, 8.48), 'text':'And I just wanted to say a word to the artist'}, {'timestamp': (8.48, 11.84), 'text':'about how beautiful and moving I thought this logo was'}, {'timestamp': (11.84, 13.72), 'text':'and your description of it.'}, {'timestamp': (13.72, 17.28), 'text':'And I think one of the things that is clear is,'}, {'timestamp': (17.28, 19.24), 'text': " you know, the words sometimes don't convey"}, {'timestamp': (19.24, 21.2), 'text':'the power of imagery or music'}, {'timestamp': (21.2, 23.12), 'text':'or the other pieces that make up culture,'}, {'timestamp': (23.12, 25.08), 'text':'and so I think the art is so important.'}, {'timestamp': (26.24, 27.76), 'text':'And when you talked about water,'}, {'timestamp': (27.76, 29.32), 'text':'I was also thinking about land,'}, {'timestamp': (29.32, 0.0), 'text': ''}, {'timestamp': (2.6, 5.04), 'text':'and I guess I just wanted to say thank you. Many of the Native American peoples of Colorado'}, {'timestamp': (5.04, 6.96), 'text':'have been at the forefront, or actually nationally,'}, {'timestamp': (6.96, 10.46), 'text':'of defending some of the public lands'}, {'timestamp': (10.46, 12.34), 'text':'that have been protected over the last few years'}, {'timestamp': (12.34, 14.32), 'text':'that are under attack right now.'}, {'timestamp': (14.32, 16.68), 'text': " And they're places that you all,"}, {'timestamp': (16.68, 18.6), 'text':'the communities have fought to protect,'}, {'timestamp': (18.6, 20.36), 'text':'but that everyone gets to enjoy.'}, {'timestamp': (20.36, 23.2), 'text': " And so I just think that it's an example"}, {'timestamp': (23.2, 26.02), 'text':'of where cultural preservation intersects'}, {'timestamp': (26.02, 29.04), 'text':'with environmental protection, with recreation'}, {'timestamp': (29.04, 0.0), 'text': ''}, {'timestamp': (3.84, 6.6), 'text':'and all of the other ways that public lands are so important. And so I think I just wanted to say thank you for that'}, {'timestamp': (6.6, 9.2), 'text':'because I think we have some very sacred places'}, {'timestamp': (9.2, 11.12), 'text':'in our country that are at risk right now.'}, {'timestamp': (11.12, 13.88), 'text':'And so as we celebrate,'}, {'timestamp': (13.88, 16.4), 'text': " I appreciate that there's still a piece of resistance"}, {'timestamp': (16.4, 20.0), 'text':'in here and I think that I just wanna mention a solidarity.'}, {'timestamp': (20.0, 22.32), 'text':'And I mentioned a feeling of solidarity'}, {'timestamp': (22.32, 23.16), 'text':'with that resistance.'}, {'timestamp': (23.16, 26.44), 'text':'So thank you and happy Confluence Week.'}, {'timestamp': (28.04, 0.0), 'text': ''}, {'timestamp': (4.24, 9.92), 'text': " Thank you, Councilman Kenich. Seeing no other comments, I'll just say a couple."}, {'timestamp': (9.92, 19.32), 'text':'In a time of such divisive ugliness and just despicable behavior from our leadership, the'}, {'timestamp': (19.32, 24.08), 'text': " reason I'm so supportive of Indigenous Peoples' Day is because it means inclusivity."}, {'timestamp': (24.08, 0.0), 'text': ''}, {'timestamp': (7.6, 10.3), 'text':'It means respecting all, respecting those who have been silenced on purpose for a long time and whose history has not been told.'}, {'timestamp': (10.3, 17.92), 'text':'And so we celebrate inclusivity in the face of such evil times, honestly.'}]}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

**Minutes of the Denver City Council Meeting**
=============================================

**Date:** Monday, October 9th
**Location:** Denver City Council Chambers
**Attendees:**

* Council Members: Lopez, Clark, Martega, Kenich, Ortega, Cashman, Canich, Sussman
* Council President: Mr. President
* City Officials: Madam Secretary Rockall, Clerk, Black, Espinosa, Flynn
* Community Members: Various

**Summary:**
The Denver City Council meeting was held on October 9th to discuss and adopt various proposals and proclamations. The main topic of the meeting was the adoption of Proclamation 1127, which observes Indigenous Peoples' Day in the city and county of Denver.

**Discussion Points:**

* Adoption of Proclamation 1127, which recognizes the contributions of Indigenous peoples to the city and county of Denver.
* Discussion of the importance of Indigenous Peoples' Day in promoting inclusivity and respect for Indigenous cultures.
* Presentation of a proclamation by Councilman Lopez, which celebrates the cultural and foundational contributions of Indigenous peoples to the city.
* Comments from Council Members, including Councilman Clark, who invited attendees to the first-ever Halloween parade on Broadway.

**Takeaways:**

* The importance of Indigenous Peoples' Day in promoting inclusivity and respect for Indigenous cultures.
* The contributions of Indigenous peoples to the city and county of Denver.
* The need for continued education and awareness about Indigenous cultures and histories.

**Action Items:**

* Adopt Proclamation 1127, which observes Indigenous Peoples' Day in the city and county of Denver.
* Support the adoption of Proclamation 1127 by all Council Members.
* Continue to promote education and awareness about Indigenous cultures and histories.

**Owners:**

* Proclamation 1127: Councilman Lopez
* Halloween Parade on Broadway: Councilman Clark<|eot_id|>